# Dataset ingestion script prototype

This notebook is intended to prototype code snippets for an ingestion script aimed at "Additional Food Bucks sites.xlsx". 

Dependencies:
* pandas
* xlrd
* os

In [1]:
import pandas as pd
import os

In [2]:
in_path = '../food-data/PFPC_data_files/Additional Food Bucks sites.xlsx'
out_path = '../food-data/Cleaned_data_files/additional_food_bucks_sites.csv'

final_cols = ['id', 'source_org', 'source_file', 'original_id', 'type', 'name', 'address', 'city', 
              'state', 'zip_code', 'county', 'location_description', 'phone', 'url', 'latitude', 
              'longitude', 'latlng_source', 'date_from', 'date_to', 'SNAP', 'WIC', 'FMNP', 
              'fresh_produce', 'food_bucks', 'free_distribution', 'open_to_spec_group', 'data_issues']

In [3]:
df = pd.read_excel(in_path)
df

,Name,Address,city,state,zip_code
0,Shop 'n Save,336 8th St,New Kensington,PA,15068.0
1,Dylamato's Market,5414 Second Avenue,Pittsburgh,PA,15207.0
2,Main Street Farmers Market,139 S Main St,Washington,PA,15301.0
3,"see Fresh Corners, Fresh Access, and Green Gro...",NaN,NaN,NaN,NaN


In [4]:
# Drop that last row of non-data
df = df.iloc[:-1, :]

# Assign some columns to schema fields
df['name'] = df['Name']
df['address'] = df['Address']

# Set some fields directly
df['source_org'] = 'The Food Trust'
df['source_file'] = os.path.basename(in_path)
# df['type'] = "farmer's market" # no consistent type - how to handle?
# df['county'] = 'Allegheny' # no consistent county - how to handle?
# df['latlng_source'] = df['source_org']
df['SNAP'] = 1 # because of food_bucks, see below
df['FMNP'] = 1
df['food_bucks'] = 1 # per Cat email - but do all FMNP farmer's markets participate in Food Bucks??
df['free_distribution'] = 0
df['data_issues'] = '' # start with blank field, to populate later

# Reorder and add any missing columns
df = df.reindex(columns = final_cols)

# Identify which columns we have handled
handled_cols = df.columns[~df.isna().all()] # i.e. columns that aren't all NA

# Detect and document missingness in handled columns
for col in handled_cols:
    df.loc[df[col].isna(), 'data_issues'] += '{} missing;'.format(col)

# Write out to CSV
df.to_csv(out_path, index = False)

df

C:\Miniconda3\envs\data-science\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Miniconda3\envs\data-science\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Miniconda3\envs\data-science\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: htt

,id,source_org,source_file,original_id,type,name,address,city,state,zip_code,...,date_from,date_to,SNAP,WIC,FMNP,fresh_produce,food_bucks,free_distribution,open_to_spec_group,data_issues
0,NaN,The Food Trust,Additional Food Bucks sites.xlsx,NaN,NaN,Shop 'n Save,336 8th St,New Kensington,PA,15068.0,...,NaN,NaN,1,NaN,1,NaN,1,0,NaN,
1,NaN,The Food Trust,Additional Food Bucks sites.xlsx,NaN,NaN,Dylamato's Market,5414 Second Avenue,Pittsburgh,PA,15207.0,...,NaN,NaN,1,NaN,1,NaN,1,0,NaN,
2,NaN,The Food Trust,Additional Food Bucks sites.xlsx,NaN,NaN,Main Street Farmers Market,139 S Main St,Washington,PA,15301.0,...,NaN,NaN,1,NaN,1,NaN,1,0,NaN,
